In [ ]:
from google.colab import drive
drive.mount('/content/drive')



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(rc={'figure.figsize': (25, 10)}, font_scale=1.5)

df = pd.read_csv('/content/drive/MyDrive/ECH_2019.csv')

# df = pd.read_csv('ECH_2019.csv')

In [ ]:
df.sample(2)



In [ ]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad', 'sueldo', 'hijos', 'barrio'])

df[mask] = df[mask].replace(no_data)



### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Seleccionamos personas con edad suficiente para trabajar

In [ ]:
df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]

In [ ]:
df = df[df['estado_laboral'].str.contains('Ocupados')]

# Categorizamos las edades y sueldos en rangos

In [ ]:
df['rango_sueldos'] = pd.cut(df['sueldo'], bins=[
    1,
    5000,
    10000,
    30000,
    50000,
    100000,
    float('Inf'),
    ]).cat.codes

df['rango_edades'] = pd.cut(df['edad'], bins=[
    0,
    10,
    15,
    20,
    30,
    40,
    50,
    60,
    70,
    float('Inf'),
    ]).cat.codes

Eliminanos las columnas que no vamos a usar o que ya categorizamos

In [ ]:
df = df[[
    'sexo', 
    'ascendencia', 
    'parentesco', 
    'estado_laboral', 
    'busca_otro_trabajo',
    'actividad',
    'asistencia alguna vez a enseñanza',
    'edu preescolar',
    'edu primaria',
    'edu media',
    'edu técnica',
    'edu univ o simil',
    'edu terciario no uni',
    'edu posgrado',
    'edu mag prof',
    'barrio',
    'estado_civil',
    'hijos',
    'leer_escribir',
    'rango_sueldos',
    'rango_edades'
    ]]

    

In [ ]:
df = df[~df['rango_sueldos'].isna()]

### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [ ]:
# Feautures
X = df.drop('rango_sueldos', axis=1).to_numpy()
y = df['rango_sueldos']

print('X shape', X.shape, '\nY shape', y.shape)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder()

X = one_hot.fit_transform(X)


In [ ]:
prediction = X.toarray()
prediction = prediction[49000].reshape(1, -1)

### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 



In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)



# DEFINIMOS ALGORITMO DE REGRESIÓN LOGISTICA

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logistic_reg = LogisticRegression(multi_class='multinomial', max_iter=20000)

logistic_reg.fit(X_train, y_train)



### VALOR DE CROSS VALIDATION PARA CADA FOLD

In [ ]:
scores = cross_val_score(logistic_reg, X, y, cv=5)

scores



In [ ]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

predictions = logistic_reg.predict(X_test)

confusion_matrix(y_test, predictions)



### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [ ]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = logistic_reg.predict(X_train)

#Prediccion en Test
y_test_pred = logistic_reg.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



In [ ]:
y_train

### APLICAMOS PREDICCIÓN SOBRE EL MODELO

In [ ]:
print(logistic_reg.predict(prediction))

print(y_train[49000])



Determinamos la probabilidad de la predicción

In [ ]:
probabilidades_prediccion = logistic_reg.predict_proba(prediction)
probabilidades_prediccion[:,1]



# REPETIMOS EL EJERCICIO CON EL MODELO DE RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(100)

random_forest.fit(X_train, y_train)



In [ ]:
scores = cross_val_score(random_forest, X, y, cv=5)

scores



In [ ]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(logistic_reg.score(X_test, y_test),2))
print('Score en train ', round(logistic_reg.score(X_train, y_train),2))



In [ ]:
predictions = logistic_reg.predict(X_test)

confusion_matrix(y_test, predictions)



In [ ]:
#Prediccion en Train
y_train_pred = random_forest.predict(X_train)

#Prediccion en Test
y_test_pred = random_forest.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



In [ ]:
print(random_forest.predict(prediction))
print(y_train[49000])



# REPETIMOS EL EJERCICIO CON EL MODELO DE K NEAREST NEIGHBOURS

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knearest_model = KNeighborsClassifier(n_neighbors=4)

knearest_model.fit(X_train, y_train)



In [ ]:
scores = cross_val_score(knearest_model, X, y, cv=5)

scores           



In [ ]:
print('Promedio de cross_validation ', round(scores.mean(),2))
print('Score en test ', round(knearest_model.score(X_test, y_test),2))
print('Score en train ', round(knearest_model.score(X_train, y_train),2))



In [ ]:
predictions = knearest_model.predict(X_test)

confusion_matrix(y_test, predictions)



In [ ]:
#Prediccion en Train
y_train_pred = knearest_model.predict(X_train)

#Prediccion en Test
y_test_pred = knearest_model.predict(X_test)

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('Aciertos sobre entrenamiento:', round(train_accuracy, 2))
print('Aciertos sobre evaluación:', round(test_accuracy, 2))



In [ ]:
print(knearest_model.predict(prediction))
print(y_train[49000])

